In [1]:
from datasets import load_from_disk
import evaluate
import multiprocessing as mp
import numpy as np
from ray import tune
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.schedulers import ASHAScheduler
import torch
from transformers import VisionEncoderDecoderModel, AutoImageProcessor, AutoTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments, default_data_collator

torch.backends.cuda.matmul.allow_tf32 = True
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device: %s" % device)

FRAMES_PER_VIDEO = 8

Device: cuda


/tmp/ipykernel_313194/738617255.py:6: DeprecationWarning: The module `ray.tune.suggest` has been moved to `ray.tune.search` and the old location will be deprecated soon. Please adjust your imports to point to the new location. Example: Do a global search and replace `ray.tune.suggest` with `ray.tune.search`.
  from ray.tune.suggest.hyperopt import HyperOptSearch
/tmp/ipykernel_313194/738617255.py:6: DeprecationWarning: The module `ray.tune.suggest.hyperopt` has been moved to `ray.tune.search.hyperopt` and the old location will be deprecated soon. Please adjust your imports to point to the new location. Example: Do a global search and replace `ray.tune.suggest.hyperopt` with `ray.tune.search.hyperopt`.
  from ray.tune.suggest.hyperopt import HyperOptSearch


In [2]:
# encoder = "facebook/timesformer-base-finetuned-k600"
# decoder = "gpt2"
# image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
# tokenizer = AutoTokenizer.from_pretrained(decoder)
# tokenizer.pad_token = tokenizer.eos_token

# def model_init(trial):
#     model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(encoder, decoder).to(device)
#     model.config.decoder_start_token_id = tokenizer.bos_token_id
#     model.config.pad_token_id = tokenizer.pad_token_id
#     return model

In [3]:
# TimeSformer
encoder = "facebook/timesformer-base-finetuned-k600"
decoder = "gpt2"

image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
tokenizer = AutoTokenizer.from_pretrained(decoder)
model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(encoder, decoder).to(device)

tokenizer.pad_token = tokenizer.eos_token
model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.pad_token_id = tokenizer.pad_token_id

# VideoMAE
# encoder = "MCG-NJU/videomae-base"
# decoder = "gpt2"

# image_processor = AutoImageProcessor.from_pretrained(encoder)
# tokenizer = AutoTokenizer.from_pretrained(decoder)
# model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(encoder, decoder).to(device)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Some weights of the model checkpoint at facebook/timesformer-base-finetuned-k600 were not used when initializing TimesformerModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing TimesformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TimesformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.6.crossattention.c_attn.weight', 'h.1.crossattention.bias', 'h.7.crossa

In [4]:
dataset = load_from_disk("/home/922201615/video-caption/dataset/processed/8frames_pt1")
dataset.set_format(type="torch")

#train_idxs = np.linspace(0, len(dataset["train"]) - 1, num=len(dataset["train"]) // 20).astype(np.int64)
#val_idxs = np.linspace(0, len(dataset["validation"]) - 1, num=len(dataset["validation"]) // 20).astype(np.int64)

train_idxs = [x for x in range(100)]
val_idxs = [x for x in range(10)]

dataset["train"] = dataset["train"].select(train_idxs)
dataset["validation"] = dataset["validation"].select(val_idxs)
dataset

DatasetDict({
    train: Dataset({
        features: ['pixel_values', 'labels'],
        num_rows: 100
    })
    validation: Dataset({
        features: ['pixel_values', 'labels'],
        num_rows: 10
    })
})

In [5]:
training_args = Seq2SeqTrainingArguments(
    output_dir="training/hp_tuning",
    predict_with_generate=True,
    tf32=True,
    dataloader_num_workers=8,
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=1,
    per_device_train_batch_size=6,
    per_device_eval_batch_size=6,
)

metric = evaluate.load("rouge")
def metrics(eval_preds):
    preds, labels = eval_preds
    preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    return metric.compute(predictions=preds, references=labels)

trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=image_processor,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    data_collator=default_data_collator,
    compute_metrics=metrics,
)

trainer.train()

# trainer = Seq2SeqTrainer(
#     model=None,
#     tokenizer=image_processor,
#     args=training_args,
#     train_dataset=dataset["train"],
#     eval_dataset=dataset["validation"],
#     data_collator=default_data_collator,
#     model_init=model_init,
# )

# def ray_hp_space(trial):
#     return {
#         "learning_rate": tune.loguniform(1e-6, 1e-3),
#         "lr_scheduler_type": tune.choice(["linear", "cosine"]),
#         "warmup_ratio": tune.uniform(0.0, 0.2),
#         "weight_decay": tune.uniform(0.0, 1e-3),
#     }

# best_trial = trainer.hyperparameter_search(
#     resources_per_trial={"cpu": mp.cpu_count(), "gpu": torch.cuda.device_count()},
#     direction="minimize",
#     backend="ray",
#     search_alg=HyperOptSearch(metric="eval_loss", mode="min"),
#     scheduler=ASHAScheduler(metric="eval_loss", mode="min"),
#     hp_space=ray_hp_space,
#     n_trials=1,
# )

# best_trial

/home/922201615/anaconda3/envs/vlm/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/home/922201615/anaconda3/envs/vlm/lib/python3.10/site-packages/torch/nn/parallel/data_parallel.py:32: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))
***** Running training *****
  Num examples = 100
  Num Epochs = 1
  Instantaneous batch size per device = 6
  Total train batch size (w. parallel, distributed & accumulation) = 30
  Gradient Accumu

OutOfMemoryError: Caught OutOfMemoryError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/922201615/anaconda3/envs/vlm/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py", line 64, in _worker
    output = module(*input, **kwargs)
  File "/home/922201615/anaconda3/envs/vlm/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1194, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/922201615/anaconda3/envs/vlm/lib/python3.10/site-packages/transformers/models/vision_encoder_decoder/modeling_vision_encoder_decoder.py", line 628, in forward
    loss = loss_fct(logits.reshape(-1, self.decoder.config.vocab_size), labels.view(-1))
  File "/home/922201615/anaconda3/envs/vlm/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1194, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/922201615/anaconda3/envs/vlm/lib/python3.10/site-packages/torch/nn/modules/loss.py", line 1174, in forward
 